In [6]:
import nltk

#nltk.download()
from nltk.corpus import wordnet as wn 
import pandas as pd

In [14]:
def findSynonyms(indicator):
    synonyms = []
    Synset = wn.synsets(indicator)

    for syns in Synset:
        #print(sysn)
        for word in syns.lemmas():
            #print(word)
            synonyms.append(word.name())
        
    return synonyms
    

def findHyponyms(indicator):
    hyponyms = []
    Synset = wn.synsets(indicator)

    for hs in Synset:
        hyponym_synset = hs.hyponyms()
        
        hyponyms = sorted(lemma.name() for synset 
            in hyponym_synset for lemma in synset.lemmas())
        
    return hyponyms

def findHypernyms(indicator):
    hypernyms = []
    Synset = wn.synsets(indicator)

    for hs in Synset:
        hypernym_synset = hs.hypernyms()
        
        hypernyms = sorted(lemma.name() for synset 
            in hypernym_synset for lemma in synset.lemmas())
        
    return hypernyms

# keywords_behvioural_intention = ['helpful', 'productive','reliabile', 'accurate']

# for word in keywords_behvioural_intention:
    
#     print(findSynonyms(word))

In [38]:
# keyword_perceived_usefulness = [helpful, timely, benificial, quick, fast, efficient, effective, productive, systematic, streamlined]
# keyword_perceived_easyofuse = [] 
# keywords_satisfaction = ['satisfied', 'useful', 'fulfill', 'gratify', 'meet', 'beneficial', 'content', 'happy', 'appeasement', "unstatisfied", "bad", "meaningless", "useless"]
# keywords_attitude = ['great', 'amazing', 'love', 'cool', 'helpful', 'dissapointed', 'useless', 'dangerous', 'disorganize', 'terrible', 'iressponsible' , 'bad']
keywords_behvioural_intention = ['infomative', 'productivereliabile', 'accurate', 'manageable', 'inaccurate', 'userfriendly', 'disorganize' , 'untrustworthy', 'uninstructive']

def findLists(keywords): 
    synonyms = []
    hyponyms = []
    hypernyms = []
    
    df_synonyms = pd.DataFrame(columns=['Word','Synonyms_list'])
    df_hyponyms = pd.DataFrame(columns=['Word','Hyponyms_list'])
    df_hypernyms = pd.DataFrame(columns=['Word','Hypernyms_list'])

    
    for i in range(len(keywords)):
        df_synonyms.loc[len(df_synonyms.index)] = [keywords[i], findSynonyms(keywords[i])]

    for i in range(len(keywords)):
        df_hyponyms.loc[len(df_hyponyms.index)] = [keywords[i], findSynonyms(keywords[i])]

    for i in range(len(keywords)):
        df_hypernyms.loc[len(df_hypernyms.index)] = [keywords[i], findSynonyms(keywords[i])]
    
    return df_synonyms,df_hyponyms, df_hyponyms

print("Behavioural Intention\n")

BI_s, BI_h, BI_hyp = findLists(keywords_behvioural_intention)
print(BI_s)
print(BI_h)
print(BI_hyp)

Behavioural Intention

                  Word                                      Synonyms_list
0           infomative                                                 []
1  productivereliabile                                                 []
2             accurate               [accurate, accurate, exact, precise]
3           manageable  [manageable, accomplishable, achievable, doabl...
4           inaccurate                                       [inaccurate]
5         userfriendly                                                 []
6          disorganize                         [disorganize, disorganise]
7        untrustworthy                          [untrustworthy, untrusty]
8        uninstructive                                    [uninstructive]
                  Word                                      Hyponyms_list
0           infomative                                                 []
1  productivereliabile                                                 []
2             a